In [1]:
# Only for google Colab
!pip install cebra
from google.colab import drive
drive.mount('/content/drive/')
!unzip /content/drive/MyDrive/Cebra_data/2020_11_9_MV1_run_brain.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.3/171.3 kB 7.5 MB/s eta 0:00:00
Mounted at /content/drive/
Archive:  /content/drive/MyDrive/Cebra_data/2020_11_9_MV1_run_brain.zip
   creating: 2020_11_9_MV1_run_brain/
  inflating: 2020_11_9_MV1_run_brain/nomove_0001_0001.tif  
  inflating: 2020_11_9_MV1_run_brain/nomove_0001_0002.tif  
  inflating: 2020_11_9_MV1_run_brain/nomove_0001_0003.tif  
  inflating: 2020_11_9_MV1_run_brain/nomove_0001_0004.tif  
  inflating: 2020_11_9_MV1_run_brain/nomove_0001_0005.tif  
  inflating: 2020_11_9_MV1_run_brain/move_0001_0001.tif  
  inflating: 2020_11_9_MV1_run_brain/move_0001_0002.tif  
  inflating: 2020_11_9_MV1_run_brain/move_0001_0003.tif  
  inflating: 2020_11_9_MV1_run_brain/move_0001_0004.tif  
  inflating: 2020_11_9_MV1_run_brain/move_0001_0005.tif  
  inflating: 2020_11_9_MV1_run_brain/move_0002_0001.tif  
  inflating: 2020_11_9_MV1_run_brai

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import cebra.datasets
from cebra import CEBRA
from PIL import Image
import cv2
import torch
import pickle

/usr/local/lib/python3.10/dist-packages/cebra/datasets/monkey_reaching.py:35: UserWarning: Could not import the nlb_tools package required for data loading of cebra.datasets.monkey_reaching. Dataset will not be available. If required, you can install the dataset by running pip install git+https://github.com/neurallatents/nlb_tools.
  warnings.warn(


In [2]:
if torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"
print(DEVICE)

cuda


In [3]:
from logging import exception
def process_brain(path):
  try:
    brain_seq = cv2.imreadmulti(path)[1]
    brain_seq = np.array(brain_seq)
    brain_mask = (np.sum(brain_seq, axis=0) > 0)
    flat_seq = (brain_seq[:, brain_mask])
    return flat_seq.astype(float)
  except:
    print(np.shape(brain_seq))


In [4]:
# iterate through all data and process it
#https://stackoverflow.com/questions/10377998/how-can-i-iterate-over-files-in-a-given-directory
import os
neural_data = []
name_data = []
behav_data = []

filepath = "2020_11_9_MV1_run_brain"
for file in os.listdir(filepath):
     filename = os.fsdecode(file)
     if filename.endswith(".tif"):
         neural_data.append(process_brain(filepath + '/' + filename))
         name_data.append(filename)
         split_name = filename.split('_')
         session_data = []
         session_data.append(int(split_name[2].split('.')[0]))
         session_data.append(int(split_name[1]))
         behav_data.append(session_data)
         continue
     else:
         continue

In [5]:
# using https://github.com/facebookresearch/dinov2
# extract feature labels from images
import torch
dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')

Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main


In [6]:
# running pretrained model: https://pytorch.org/docs/stable/hub.html
dir(dinov2_vits14)


['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_intermediate_layers_chunked',
 '_get_intermediate_layers_not_chunked',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffer

In [7]:
help(dinov2_vits14.train)

Help on method train in module torch.nn.modules.module:

train(mode: bool = True) -> ~T method of dinov2.models.vision_transformer.DinoVisionTransformer instance
    Sets the module in training mode.
    
    This has any effect only on certain modules. See documentations of
    particular modules for details of their behaviors in training/evaluation
    mode, if they are affected, e.g. :class:`Dropout`, :class:`BatchNorm`,
    etc.
    
    Args:
        mode (bool): whether to set training mode (``True``) or evaluation
                     mode (``False``). Default: ``True``.
    
    Returns:
        Module: self



In [43]:

#generate mock behavioural data for time only run
mock_behav = []
for count, data in enumerate(neural_data):
    mock_behav.append(np.arange(np.size(data[:, 0]), dtype=float))


In [38]:
out_dim = 8
max_iterations = 10

In [39]:
multi_cebra_model = cebra.CEBRA(batch_size=1,
                                output_dimension=out_dim,
                                max_iterations=200,
                                verbose=True,
                                max_adapt_iterations=10)

In [44]:
multi_cebra_model.fit(neural_data[:50], mock_behav[:50])

pos:  0.8668 neg:  3.0214 total:  3.8882 temperature:  1.0000: 100%|██████████| 200/200 [01:06<00:00,  3.00it/s]


CEBRA(batch_size=1, max_adapt_iterations=10, max_iterations=200, verbose=True)

In [12]:
print(np.shape(mock_behav[1]))
np.shape(neural_data[1])

(101,)


(101, 10634)

In [ ]:


fig = plt.figure()

ax1 = plt.subplot(141, projection='3d')

for run, data in enumerate(neural_data[:50]):
  embedding = multi_cebra_model.transform(data, session_id=run)
  ax1 = cebra.plot_embedding(embedding, ax=ax1)

plt.show()


In [4]:
!ls -R /* | grep ".ipynb"

ls: reading directory '/proc/51/net': Invalid argument
ls: reading directory '/proc/51/task/51/net': Invalid argument
index.ipynb
Categorical.ipynb
Continuous.ipynb
index.ipynb
index.ipynb
flax_functional_engine.ipynb
area_chart.ipynb
autompg_histogram.ipynb
autompg_violins.ipynb
bachelors_degrees_by_gender.ipynb
bars_economic.ipynb
box_draw_roi_editor.ipynb
boxplot_chart.ipynb
choropleth_data_link.ipynb
directed_airline_routes.ipynb
dot_example.ipynb
dragon_curve.ipynb
dropdown_economic.ipynb
emoji_tsne.ipynb
energy_sankey.ipynb
hextile_movie_ratings.ipynb
histogram_example.ipynb
image_range_tool.ipynb
iris_density_grid.ipynb
iris_example.ipynb
iris_grouped_grid.ipynb
iris_splom_example.ipynb
irregular_quadmesh.ipynb
legend_example.ipynb
lesmis_example.ipynb
life_expectancy_split_violin.ipynb
lorenz_attractor_example.ipynb
mandelbrot_section.ipynb
measles_example.ipynb
network_graph.ipynb
nyc_airport_connections.ipynb
nyc_radial_heatmap.ipynb
point_draw_triangulate.ipynb
quiver_demo.i